In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
nltk.download()
from nltk.corpus import stopwords
from string import punctuation
from nltk.tokenize import word_tokenize
from nltk.stem import LancasterStemmer

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> all


       | 
       | Downloading package abc to /root/nltk_data...
       |   Package abc is already up-to-date!
       | Downloading package alpino to /root/nltk_data...
       |   Package alpino is already up-to-date!
       | Downloading package averaged_perceptron_tagger to
       |     /root/nltk_data...
       |   Package averaged_perceptron_tagger is already up-to-date!
       | Downloading package averaged_perceptron_tagger_ru to
       |     /root/nltk_data...
       |   Package averaged_perceptron_tagger_ru is already up-to-
       |       date!
       | Downloading package basque_grammars to /root/nltk_data...
       |   Package basque_grammars is already up-to-date!
       | Downloading package biocreative_ppi to /root/nltk_data...
       |   Package biocreative_ppi is already up-to-date!
       | Downloading package bllip_wsj_no_aux to /root/nltk_data...
       |   Package bllip_wsj_no_aux is already up-to-date!
       | Downloading package book_grammars to /root/nltk_data..


---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv('/content/drive/My Drive/NLP datasets/Consumer_Complaints_big.csv')
df.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (5,6,11,16) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,07/17/2019,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Account status incorrect,NaN,NaN,ALLY FINANCIAL INC.,TX,75035,NaN,NaN,Web,07/17/2019,In progress,Yes,NaN,3309495
1,07/17/2019,Credit card or prepaid card,General-purpose prepaid card,Trouble using the card,Problem using the card to withdraw money from ...,NaN,NaN,SQUARE INC,KS,674XX,NaN,NaN,Web,07/17/2019,In progress,Yes,NaN,3310031
2,07/17/2019,Debt collection,Other debt,Took or threatened to take negative or legal a...,Threatened or suggested your credit would be d...,NaN,NaN,"Diversified Consultants, Inc.",FL,NaN,NaN,NaN,Web,07/17/2019,In progress,Yes,NaN,3309687
3,07/17/2019,Mortgage,VA mortgage,Trouble during payment process,NaN,NaN,NaN,"FLAGSTAR BANK, FSB",VA,22554,Servicemember,NaN,Web,07/17/2019,In progress,Yes,NaN,3308925
4,07/17/2019,Debt collection,Other debt,Attempts to collect debt not owed,Debt is not yours,NaN,Company believes it acted appropriately as aut...,BYL Collection Services,TN,370XX,Servicemember,NaN,Web,07/17/2019,Closed with explanation,Yes,NaN,3308914


In [ ]:
df.shape

(1334876, 18)

In [ ]:
df1 = df[['Product','Consumer complaint narrative']]

In [ ]:
df1['Product'].value_counts()

Mortgage                                                                        282345
Debt collection                                                                 253853
Credit reporting, credit repair services, or other personal consumer reports    252092
Credit reporting                                                                140432
Credit card                                                                      89190
Bank account or service                                                          86206
Student loan                                                                     52889
Credit card or prepaid card                                                      52468
Checking or savings account                                                      45000
Consumer Loan                                                                    31604
Vehicle loan or lease                                                            12408
Money transfer, virtual currency, or money 

In [ ]:
df1['Product'] = df1['Product'].str.lower()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
df1.isnull().sum()

Product                              0
Consumer complaint narrative    938347
dtype: int64

In [ ]:
df1.shape

(1334876, 2)

In [ ]:
df1= df1.dropna()

In [ ]:
df1.shape

(396529, 2)

In [ ]:
def renaming(x):
   #print(x)
   if x =='credit reporting':
        x = 'credit reporting, credit repair services, or other personal consumer reports'
   elif x == 'credit card':
        x = 'credit card or prepaid card'
   elif x == 'virtual currency':
        x = 'money transfer, virtual currency, or money service'
   elif x == 'prepaid card':
        x = 'credit card or prepaid card'
   elif x == 'money transfers':
        x = 'money transfer, virtual currency, or money service'
   elif x == 'student loan': 
        x='Loans'
   elif x=='consumer loan': 
        x='Loans'
   elif x=='payday loan': 
        x='Loans'
   elif x=='checking or savings account': 
        x='bank account or service'
   elif x=='payday loan, title loan, or personal loan': 
        x='Loans'
   elif x=='vehicle loan or lease': 
        x='Loans'
   return x

In [ ]:
df1['Product'] = df1['Product'].apply(renaming)

In [ ]:
df1['Product'].value_counts()

credit reporting, credit repair services, or other personal consumer reports    129418
debt collection                                                                  89379
mortgage                                                                         54109
Loans                                                                            44409
credit card or prepaid card                                                      43103
bank account or service                                                          28554
money transfer, virtual currency, or money service                                7265
other financial service                                                            292
Name: Product, dtype: int64

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df1['Product'] = le.fit_transform(df1['Product'])

In [ ]:
value_counts = df1['Product'].value_counts()
value_counts

3    129418
4     89379
6     54109
0     44409
2     43103
1     28554
5      7265
7       292
Name: Product, dtype: int64

In [ ]:
df1 = df1[~df1['Product'].isin(value_counts[value_counts<10000].index)]
df1['Product'].value_counts()

3    129418
4     89379
6     54109
0     44409
2     43103
1     28554
Name: Product, dtype: int64

In [ ]:
# from string import punctuation
# from nltk.corpus import stopwords
stuff_to_be_removed = list(stopwords.words('english'))+list(punctuation)
from nltk.tokenize import word_tokenize
from nltk.stem import LancasterStemmer
stemmer = LancasterStemmer()
corpus = df1['Consumer complaint narrative'].tolist()
len(corpus)
corpus = corpus[1:10000]
print(corpus[1])

When they call they dont answer the call and they call all day long. I havent sign any contact with this company I never had any comunicacin with agency! I never sign any contract or made any commitment with Maury Cobb & Associates! As a consumer by the law on the credit company if I did not sign any contact with this collect this need to be removed from my XXXX report!


In [ ]:
final_corpus = []
for i in range(len(corpus)):
  word = word_tokenize(corpus[i].lower())
  word = [stemmer.stem(y) for y in word if y not in stuff_to_be_removed]
  j = " ".join(word)
  final_corpus.append(j)
final_corpus[1]

'cal dont answ cal cal day long hav sign contact company nev comunicacin ag nev sign contract mad commit maury cob assocy consum law credit company sign contact collect nee remov xxxx report'

In [ ]:
product_list=df1['Product'].tolist()
product_list=product_list[1:10000]
newdf = pd.DataFrame(final_corpus)
newdf['label'] = product_list
newdf.columns=['sentence','label']
newdf.head()

,sentence,label
0,16000.00 debt,3
1,cal dont answ cal cal day long hav sign contac...,4
2,provid account inform show debt paid credit ka...,4
3,2 charg credit report collect ag `` trid asset...,4
4,xxxx xxxx xxxx xxxx post 3 collect account 3 c...,3


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
vector = tfidf.fit_transform(newdf['sentence'])
print(type(vector))
x_train = vector.toarray()
print(x_train)
y_train= newdf['label']

<class 'scipy.sparse.csr.csr_matrix'>
[[0.27715548 0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.20051905 0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.05226919 0.         0.         ... 0.         0.         0.        ]]


In [ ]:
from sklearn.naive_bayes import MultinomialNB
ab = MultinomialNB()
ab.fit(x_train,y_train)
ypred = ab.predict(x_train)
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
print(accuracy_score(y_train,ypred))
print(confusion_matrix(y_train,ypred))
print(classification_report(y_train,ypred))

0.6332633263326333
[[ 258    0    3  627  113    0]
 [   0   38   63  467   34    0]
 [   0    0  261  807   43    0]
 [   0    3    0 3971   73    1]
 [   3    0    3  835 1549    1]
 [   5    0    0  540   46  255]]
              precision    recall  f1-score   support

           0       0.97      0.26      0.41      1001
           1       0.93      0.06      0.12       602
           2       0.79      0.23      0.36      1111
           3       0.55      0.98      0.70      4048
           4       0.83      0.65      0.73      2391
           6       0.99      0.30      0.46       846

    accuracy                           0.63      9999
   macro avg       0.84      0.41      0.46      9999
weighted avg       0.75      0.63      0.59      9999



In [ ]:
from sklearn.tree import DecisionTreeClassifier
dtree_model = DecisionTreeClassifier(max_depth = 25)
dtree_model.fit(x_train, y_train)
ypred2 = dtree_model.predict(x_train)
print(accuracy_score(y_train,ypred2))
print(classification_report(y_train,ypred2))

0.9526952695269527
              precision    recall  f1-score   support

           0       0.96      0.94      0.95      1001
           1       0.96      0.93      0.94       602
           2       0.94      0.93      0.93      1111
           3       0.94      0.98      0.96      4048
           4       0.96      0.93      0.95      2391
           6       0.99      0.94      0.96       846

    accuracy                           0.95      9999
   macro avg       0.96      0.94      0.95      9999
weighted avg       0.95      0.95      0.95      9999



In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 7)
knn.fit(x_train, y_train)
ypred4 = knn.predict(x_train)
print(accuracy_score(y_train,ypred4))
print(classification_report(y_train,ypred4))

0.7815781578157815
              precision    recall  f1-score   support

           0       0.80      0.66      0.72      1001
           1       0.78      0.73      0.76       602
           2       0.76      0.71      0.74      1111
           3       0.74      0.93      0.82      4048
           4       0.85      0.67      0.75      2391
           6       0.90      0.68      0.78       846

    accuracy                           0.78      9999
   macro avg       0.81      0.73      0.76      9999
weighted avg       0.79      0.78      0.78      9999



In [ ]:
from sklearn.ensemble import RandomForestClassifier
rdforest = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 42)
rdforest.fit(x_train, y_train)
ypred5 = rdforest.predict(x_train)
print(accuracy_score(y_train,ypred5))
print(classification_report(y_train,ypred5))

0.9956995699569957
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1001
           1       1.00      1.00      1.00       602
           2       0.99      0.99      0.99      1111
           3       1.00      1.00      1.00      4048
           4       1.00      0.99      1.00      2391
           6       1.00      0.99      0.99       846

    accuracy                           1.00      9999
   macro avg       1.00      0.99      1.00      9999
weighted avg       1.00      1.00      1.00      9999



In [ ]:
from sklearn import svm
lin_clf = svm.LinearSVC()
lin_clf.fit(x_train, y_train)
ypred3 = lin_clf.predict(x_train)
print(accuracy_score(y_train,ypred3))
print(classification_report(y_train,ypred3))

0.953995399539954
              precision    recall  f1-score   support

           0       0.95      0.93      0.94      1001
           1       0.97      0.99      0.98       602
           2       0.96      0.95      0.95      1111
           3       0.96      0.96      0.96      4048
           4       0.94      0.94      0.94      2391
           6       0.98      0.98      0.98       846

    accuracy                           0.95      9999
   macro avg       0.96      0.96      0.96      9999
weighted avg       0.95      0.95      0.95      9999



In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
xgb = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)
xgb.fit(x_train, y_train)
ypred6 = xgb.predict(x_train)
print(accuracy_score(y_train,ypred6))
print(classification_report(y_train,ypred6))

KeyboardInterrupt: ignored